<a href="https://colab.research.google.com/github/bhatiap70077/Natural-Language-Processing/blob/main/Word_Vect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk 
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
import re
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# Read the data
a=[]
b=[]
with open('airline_review_training_data.csv','r') as f:
    l=f.readlines()
    for j in l:
        a.append(j.split('\t')[0])
        b.append(j.split('\t')[0])

In [ ]:
# convert to pandas 
d={'label':a,'document':b}
df=pd.DataFrame(d)
df.head()

,label,document
0,"text,airline_sentiment\n","text,airline_sentiment\n"
1,"""@USAirways BF has been stuck in CLT all day. ...","""@USAirways BF has been stuck in CLT all day. ..."
2,@united DM sent. This lack if customer servic...,@united DM sent. This lack if customer servic...
3,"@JetBlue thank you for the information.,1\n","@JetBlue thank you for the information.,1\n"
4,@AmericanAir I DMed you my AA &amp; phone #s &...,@AmericanAir I DMed you my AA &amp; phone #s &...


In [ ]:
# taking a sample of data
sample_data = df.tail(574)

In [ ]:
# get stop words
# from nltk.corpus import stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# lower case

sample_data1 = (
    # start with training data
    sample_data
    # convert to lower case
    .assign(clean_document= lambda x:[text.lower() for text in x.document]) 
    # remove punctuation
    .assign(clean_document= lambda x: [re.sub("[^\w\s]","",text) for text in x.clean_document])
     # remove numbers
    .assign(clean_document= lambda x: [re.sub("\d","",text) for text in x.clean_document])
    # remove leading & trainling spaces 
    .assign(clean_document = lambda x: [text.strip() for text in x.clean_document])
    # remove all the '\n' values with space
    .assign(clean_document= lambda x: [re.sub(r"\n","",text) for text in x.clean_document] )
    # word tokenize
    .assign(word_token= lambda x: [nltk.word_tokenize(text) for text in x.clean_document] )
    # # remove stop words
    .assign(word_token= lambda x: [list(set(list_of_words).difference(stop_words)) for list_of_words in x.word_token])
    # Lemmatising the words
    # .assign(word_token= lambda x: [[lem.lemmatize(a_word) for a_word in list_of_words] for list_of_words in x.word_token ])
    #  # input to sklearn
    .assign(document_to_sklearn= lambda x: [" ".join(map(str,list_of_words)) for list_of_words in x.word_token ])
)

In [ ]:
from collections import defaultdict

def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)  # add to vocab
            next_token = text[i+1 : i+1+window_size]
            for t in next_token:
                key = tuple( sorted([t, token]) )
                d[key] += 1
    
    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [ ]:
text = sample_data1['document_to_sklearn']
df = co_occurrence(text, 3)
df

,aa,abc_wtvd,ability,able,absolutely,abysmal,accept,access,accessible,accomplish,...,yet,yo,youd,youll,youre,youve,yr,yrs,yyz,zoom
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abc_wtvd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
absolutely,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
youve,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yr,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yrs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yyz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(5)
pca_d = pca.fit_transform(df)
df1 = pd.DataFrame(pca_d,index=df.index,columns=['pca1','pca2','pca3','pca4','pca5'])

In [ ]:
df1['words'] = df1.index

In [ ]:
import plotly.express as px

In [ ]:
df1.index

Index(['aa', 'abc_wtvd', 'ability', 'able', 'absolutely', 'abysmal', 'accept',
       'access', 'accessible', 'accomplish',
       ...
       'yet', 'yo', 'youd', 'youll', 'youre', 'youve', 'yr', 'yrs', 'yyz',
       'zoom'],
      dtype='object', length=2011)

In [ ]:
df1

,pca1,pca2,pca3,pca4,pca5,words
aa,2.556606,0.412454,1.609940,0.717828,-0.012680,aa
abc_wtvd,-0.523775,0.237294,-0.221715,-0.070503,0.412564,abc_wtvd
ability,-0.836055,-0.135856,-0.131399,0.164104,0.015831,ability
able,2.482441,1.082723,0.977731,-1.460928,1.040767,able
absolutely,-0.465802,-0.292858,0.021355,0.220626,0.576895,absolutely
...,...,...,...,...,...,...
youve,-0.518623,0.185960,-0.222343,-0.070309,0.483016,youve
yr,-0.564330,0.209760,-0.199782,-0.109720,0.425490,yr
yrs,-0.838008,-0.164013,-0.080691,0.224267,0.024091,yrs
yyz,-0.252521,0.470945,-0.133376,-0.080724,0.607296,yyz


In [ ]:
d ={}
x = np.array(df.loc['customers'])

for word in df.index:
  y = np.array(df.loc[word])

  d[word]=np.linalg.norm(x - y)

euclidian= pd.DataFrame.from_dict(d, orient='index')

euclidian.sort_values(by=[0],inplace=True)

In [ ]:
euclidian.head(10)

,0
customers,0.000000
choose,4.898979
colors,4.898979
serves,5.000000
meal,5.000000
gooutofbusiness,5.000000
apologies,5.099020
pmtold,5.099020
hopefully,5.099020
oy,5.099020


In [ ]:
import plotly.express as px

In [ ]:
df1.index

Index(['aa', 'abc_wtvd', 'ability', 'able', 'absolutely', 'abysmal', 'accept',
       'access', 'accessible', 'accomplish',
       ...
       'yet', 'yo', 'youd', 'youll', 'youre', 'youve', 'yr', 'yrs', 'yyz',
       'zoom'],
      dtype='object', length=2011)

In [ ]:
fig = px.scatter( x=df1["pca4"], y=df1["pca5"],color=df.index)
fig.show()

In [ ]:
# build a function, that , given a word , returns the closest word....

In [ ]:
#Eucledian Distance

In [ ]:

from scipy.spatial import distance

In [ ]:
d ={}
x = np.array(df.loc['sorry'])

for word in df.index:
  y = np.array(df.loc[word])
  d[word]=np.linalg.norm(x - y)
  euclidian= pd.DataFrame.from_dict(d, orient='index')
  euclidian.sort_values(by=[0],inplace=True)




output = {}
output['Euclidean'] = euclidian.head().to_dict()[0]

    

In [ ]:
euclidian.head(5)

,0
sorry,0.000000
ppl,4.242641
fault,4.358899
wearing,4.358899
traveling,4.358899


In [ ]:
#Cosine Similarity

In [ ]:
e ={}
t1 = np.array(df.loc['sorry'])

for word in df1.index:
  t2 = np.array(df.loc[word])
  e[word] = t1.dot(t2)/ (np.linalg.norm(t1, axis=0) * np.linalg.norm(t2))
  cos_val = pd.DataFrame.from_dict(e, orient='index')
  cos_val.sort_values(by=[0],inplace=True)

In [ ]:
cos_val

,0
lbs,0.000000
mom,0.000000
mobile,0.000000
mllovelace,0.000000
miserably,0.000000
...,...
depart,0.300000
failure,0.300000
im,0.306286
ppl,0.365148


In [ ]:
cos_val.head(5)

,0
lbs,0.0
mom,0.0
mobile,0.0
mllovelace,0.0
miserably,0.0


## **NOTE**

In [ ]:
NOTE: """I calculated eucledian dist between dist of sound word's pca1 value to pca2 values. Now I want to save this output as a column with our dataset. Then we need to arrange the dataset in decreasing order of eucledian dist column and find first 5 values
and their assigned words.

Now work on saving this column with dataset, arranging dataset into dec order based on this result value, and find top 5 values. Same thing we have to do for cosine dist also.


Link1: https://www.delftstack.com/howto/python/cosine-similarity-between-lists-python/#:~:text=of%20equal%20sizes.-,Use%20the%20scipy%20Module%20to%20Calculate%20the%20Cosine%20Similarity%20Between,of%20the%20distance%20from%201.

Link2: https://www.w3schools.com/python/python_functions.asp



Link3: https://vedexcel.com/how-to-calculate-euclidean-distance-in-python/



Link4: https://exerror.com/how-to-calculate-euclidean-distance-in-python/



Link5: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

"""

In [ ]:
def find_match(word:str)->dict: 
    # Creating dicts for distance calculations
    euclidean_dist = {}
    cosine_dist = {}
    
    # getting the dimensions for word
    a = np.array(df1.loc[word])
    
    # looping through vocab and measuring distance 
    for item in df1.index:
        b = np.array(df1.loc[item])
        euclidean_dist[item] = distance.euclidean(int(a), int(b))
        cosine_dist[item] = distance.cosine(int(a), int(b))
    
    # Sorting and creating dataframe
    euclidean = pd.DataFrame.from_dict(euclidean_dist, orient='index').sort_values(by=[0])
    cosine = pd.DataFrame.from_dict(cosine_dist, orient='index').sort_values(by=[0])
    
    # Giving Top 3 from both distances 
    output = {}
    output['Euclidean'] = euclidean.head().to_dict()[0]
    output['Cosine'] = cosine.head().to_dict()[0]
    
    return(output)
